In [1]:
import glob
import torch
import numpy as np

from cc.model import Classifier
from cc.dataset import CCDataset

import caml_core as core

import torchcam

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
DATASET_PATH = "data/v3"
MODEL = "version_0"

In [3]:
checkpoint_path = glob.glob(f"lightning_logs/{MODEL}/checkpoints/*")[0]
checkpoint = torch.load(checkpoint_path)

model = Classifier().eval()
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [4]:
validation_dataset = CCDataset(path=DATASET_PATH, type="val")
validation_dataloader = torch.utils.data.DataLoader(
        dataset=validation_dataset,
        batch_size=1,
        shuffle=True,
        num_workers=0,
        drop_last=False
    )

In [5]:
# for sample in validation_dataloader:
#     x, y = sample

#     bolt_gt = y[0, 0].cpu().item()
#     hinge_gt = y[0, 1].cpu().item()

#     bolt_logits, hinge_logits = model.forward(x=x)
#     if bolt_logits is not None:
#         bolt_probs = torch.sigmoid(bolt_logits).detach().cpu().item()
#     else:
#         bolt_probs = None

#     if hinge_logits is not None:
#         hinge_probs = torch.sigmoid(hinge_logits).detach().cpu().item()
#     else:
#         hinge_probs = None

#     if (bolt_probs is not None  and (bolt_gt != np.round(bolt_probs)) or (hinge_probs is not None and hinge_gt != np.round(hinge_probs))):
#         print(f"label: Bolt - {bolt_gt} \t Hinge - {hinge_gt}")
#         print(f"Pred: Bolt - {bolt_probs} \t Hinge - {hinge_probs}")

#         image = x[0].detach().cpu().numpy().astype(np.uint8)
#         image = core.geometry.Image(image=image.transpose((1, 2, 0)))
#         h, w, _ = image.get_dimensions()
#         image = image.resize((3 * h, 3 * w))
        
#         image.show()

In [6]:
cam = torchcam.methods.XGradCAM(model, '_additional_conv')

for index, sample in enumerate(validation_dataloader):
    x, y = sample

    bolt_gt = y[0, 0].cpu().item()
    hinge_gt = y[0, 1].cpu().item()

    # For CAM Visualization
    new_x = model.pre_processor(x)
    new_x.requires_grad = True

    bolt_logits, hinge_logits = model.forward(x=new_x, skip_pre_process=True)
    if bolt_logits is not None:
        bolt_probs = torch.sigmoid(bolt_logits)
    else:
        bolt_probs = None

    if hinge_logits is not None:
        hinge_probs = torch.sigmoid(hinge_logits)
    else:
        hinge_probs = None

    if (bolt_probs is not None  and (bolt_gt != np.round(bolt_probs.detach().cpu().item())) or (hinge_probs is not None and hinge_gt != np.round(hinge_probs.detach().cpu().item()))):

        print(f"label: Bolt - {bolt_gt} \t Hinge - {hinge_gt}")
        print(f"Pred: Bolt - {bolt_probs.detach().cpu().item()} \t Hinge - {hinge_probs.detach().cpu().item()}")

        image = x[0].detach().cpu().numpy().astype(np.uint8)
        image = core.geometry.Image(image=image.transpose((1, 2, 0)))

        class_id = int(torch.round(hinge_probs).item())
        scores = torch.cat([ 1- hinge_probs, hinge_probs], dim=-1)
        act_map = cam(class_idx=class_id, scores=scores)

        act_map = act_map[0][0].detach().cpu().numpy().astype(float)
        act_map = core.geometry.Image(act_map).resize(image.get_dimensions()[:2])

        # Attenuate activation map
        act_map.image = act_map.image * act_map.image

        # Overlay activation map on the image
        overlaid_image = (act_map.image * 0.85)[..., None] + (image.image / 255.0) * 0.15
        overlaid_image = core.geometry.Image(overlaid_image)
        overlaid_image = overlaid_image.resize((448, 448))
        overlaid_image.show()


label: Bolt - 0.0 	 Hinge - 0.0
Pred: Bolt - 0.6192986965179443 	 Hinge - 0.004175654146820307


Gtk-Message: 10:22:15.635: Failed to load module "canberra-gtk-module"


label: Bolt - 1.0 	 Hinge - 0.0
Pred: Bolt - 0.1944405883550644 	 Hinge - 0.003031122963875532
